In [1]:
! pip install transformers
! pip install datasets
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 8.6 MB/s 
     |████████████████████████████████| 163 kB 66.9 MB/s 
     |████████████████████████████████| 7.6 MB 54.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 6.7 MB/s 
     |████████████████████████████████| 115 kB 51.8 MB/s 
     |████████████████████████████████| 212 kB 60.3 MB/s 
     |████████████████████████████████| 127 kB 58.9 MB/s 
     |████████████████████████████████| 115 kB 47.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

In [50]:
from transformers import BertTokenizer, BertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained('susumu2357/bert-base-swedish-squad2')
model = BertForQuestionAnswering.from_pretrained('susumu2357/bert-base-swedish-squad2')

Downloading:   0%|          | 0.00/399k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [20]:
from datasets import load_dataset
dataset = load_dataset("susumu2357/squad_v2_sv", split='train' )


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/113898 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11156 [00:00<?, ? examples/s]

Dataset squad_v2_sv downloaded and prepared to /root/.cache/huggingface/datasets/susumu2357___squad_v2_sv/squad_v2/2.0.0/a9da0fd2d29111d7f363bae7e7563712194259a87dd3f5e3773af0829f3c673a. Subsequent calls will reuse this data.


In [51]:
len(dataset)

113898

In [52]:
print(dataset)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 113898
})


In [53]:
dataset.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [54]:
dataset[0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ / bee-YON-say) (född 4 september 1981) är en amerikansk sångerska, låtskrivare, skivproducent och skådespelerska. Född och uppvuxen i Houston, Texas, uppträdde hon i olika sång- och danstävlingar som barn och blev berömmelse i slutet av 1990-talet som sångare för R & B-flickgruppen Destiny\'s Child. Gruppen hanterades av sin far Mathew Knowles och blev en av världens bästsäljande tjejgrupper genom tiderna. I deras avbrott släpptes Beyoncés debutalbum, Dangerously in Love (2003), som etablerade henne som en soloartist över hela världen, tjänade fem Grammy Awards och presenterade Billboard Hot 100-singlarna "Crazy in Love" och "Baby Boy" .',
 'question': 'När började Beyonce bli populär?',
 'answers': {'text': ['i slutet av 1990-talet'], 'answer_start': [270]}}

In [55]:
# hur ser vår datamängd ut
print("The number of rows", dataset.num_rows, "also available as len(dataset)", len(dataset))
print("The number of columns", dataset.num_columns)
print("The shape (rows, columns)", dataset.shape)

The number of rows 113898 also available as len(dataset) 113898
The number of columns 5
The shape (rows, columns) (113898, 5)


In [56]:
print(dataset['question'][:10])

['När började Beyonce bli populär?', 'Vilka områden tävlade Beyonce på när hon växte upp?', "När lämnade Beyonce Destiny's Child och blev solosångare?", 'I vilken stad och delstat växte Beyonce upp?', 'Under vilket årtionde blev Beyonce känd?', 'I vilken R & B-grupp var hon sångare?', 'Vilket album gjorde henne till en världskänd artist?', "Vem hanterade gruppen Destiny's Child?", 'När blev Beyoncé berömd?', "Vilken roll hade Beyoncé i Destiny's Child?"]


In [57]:
# Förbered datamängden för svensk BERT 
def convert_to_features(batch):
    # Tokenize parvisa input is frågor och svar 
    input_pairs = list(zip())
    encodings = tokenizer(batch['context'], batch['question'], truncation=True)

    # Beräkna start och slutpostioner (här får jag det tyvärr inte att fungera än så länge, kan modellerna vara för gamla?)
    start_positions, end_positions = [], []
    for i, answer in enumerate(batch['answers']):
        first_char = answer['answer_start'][0]
        last_char = first_char + len(answer['text'][0]) - 1
        start_positions.append(encodings.(i, first_char))
        end_positions.append(encodings.char_to_token(i, last_char))

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

encoded_dataset = dataset.map(convert_to_features, batched=True)

  0%|          | 0/114 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: ignored

In [24]:
def get_correct_alignement(context, answer):
    """ Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here. """
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(gold_text)
    if context[start_idx:end_idx] == gold_text:
        return start_idx, end_idx       # When the gold label position is good
    elif context[start_idx-1:end_idx-1] == gold_text:
        return start_idx-1, end_idx-1   # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        return start_idx-2, end_idx-2   # When the gold label is off by two character
    else:
        raise ValueError()

In [25]:
# Tokenize our training dataset
def convert_to_features(example_batch):
    # Tokenize contexts and questions (as pairs of inputs)
    encodings = tok(example_batch['context'], example_batch['question'], truncation=True)

    # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methods.
    start_positions, end_positions = [], []
    for i, (context, answer) in enumerate(zip(example_batch['context'], example_batch['answers'])):
        start_idx, end_idx = get_correct_alignement(context, answer)
        start_positions.append(encodings.char_to_token(i, start_idx))
        end_positions.append(encodings.char_to_token(i, end_idx-1))
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

encoded_dataset = dataset.map(convert_to_features, batched=True)

  0%|          | 0/114 [00:00<?, ?ba/s]

IndexError: ignored